In [ ]:
import pandas as pd
import yfinance as yf
from datetime import timedelta
from datetime import datetime
from pandas_market_calendars import get_calendar
from tqdm.auto import tqdm

## Use Alpha and Beta to calculate ER

In [3]:
file = r"event_data_for_abnormal_return.xlsx"
df = pd.read_excel(file)
df.dropna(how='all', inplace=True)
df.head(1)

,nid,Eventdate,Ticker,Estimate_event_date_start,Unnamed: 4,alpha,beta
0,1.0,2021-03-25,WM,2020-08-31,NaN,0.000097,0.633873


In [ ]:
import time 

event_ar_matrix = []

for nid, event_date, ticker, alpha, beta in zip(df["nid"], df["Eventdate"], df["Ticker"], df["alpha"], df["beta"]):
    event_date = datetime.strptime(event_date, "%Y-%m-%d")

    company_return_list = []
    sp_return_list = []

    # Calculate different date after event happned
    nyse = get_calendar("NYSE")

    # Get trading day
    trading_days = nyse.schedule(
        start_date=event_date - timedelta(days=30),  
        end_date=event_date + timedelta(days=30)     
    ).index

    # Need the trading day right before Event Day + 1
    trading_days_after_event = trading_days[trading_days > event_date][:10]
    index_list = list(trading_days.get_indexer_for(trading_days_after_event))
    index_list.append(trading_days.get_loc(trading_days_after_event[0]) - 1)
    index_list.sort()
    trading_days_after_event_with_one_day_before_it = trading_days[index_list]

    # Fetch one
    company = yf.Ticker(ticker)
    sp = yf.Ticker("^GSPC")

    # Run in different day
    for index in range(10):
        date_last = trading_days_after_event_with_one_day_before_it[index]
        event_date_adjust_history = trading_days_after_event_with_one_day_before_it[index + 1] + timedelta(days=1) # history's end para is exclusive

        time.sleep(3)

        # Get Company's event date return
        df_company = company.history(start=date_last, end=event_date_adjust_history, auto_adjust=True)
        company_return = df_company["Close"].pct_change()
        company_return.dropna(inplace=True)

        time.sleep(3)

        # Get S&P500 event date return
        df_sp = sp.history(start=date_last, end=event_date_adjust_history, auto_adjust=True)  
        sp_return = df_sp["Close"].pct_change() # calculate Return for each day
        sp_return.dropna(inplace=True)

    
        if not company_return.empty and not sp_return.empty:
            company_return_list.append(company_return)
            sp_return_list.append(sp_return)
        

    # # Only past 10 days are needed 
    company_return_s = pd.concat(company_return_list)
    sp_return_s = pd.concat(sp_return_list)

    df_return = pd.DataFrame({
        "Company_return":company_return_s,
        "SP500_return":sp_return_s
    })
    

    # # Calcualte AR for each trade day
    abnormal_return_list = []
    for day_number in range(10):
        company_return_data = df_return.iloc[day_number, 0]
        sp500_return_data = df_return.iloc[day_number, 1]

    #     # Expected Return
        expected_return = alpha + beta * sp500_return_data

    #     # Abnormal Return
        abnormal_return = company_return_data - expected_return

        abnormal_return_list.append(abnormal_return)

    event_ar_matrix.append(abnormal_return_list)

    print(f'{nid}完成')


# column name
ar_columns = [f"AR{i+1}" for i in range(len(abnormal_return_list))]

# # dataframe
df_ar_matrix = pd.DataFrame(event_ar_matrix, columns=ar_columns)

In [ ]:
event_ar_matrix = []


for nid, event_date, ticker, alpha, beta in tqdm(zip(df["nid"], df["Eventdate"], df["Ticker"], df["alpha"], df["beta"]), desc="進度"):
    event_date = datetime.strptime(event_date, "%Y-%m-%d")

    company_return_list = []
    sp_return_list = []

    # Calculate different date after event happned
    nyse = get_calendar("NYSE")

    # Get trading day
    trading_days = nyse.schedule(
        start_date=event_date - timedelta(days=30),  
        end_date=event_date + timedelta(days=30)     
    ).index

    # Need the trading day right before Event Day + 1
    trading_days_before_event = trading_days[trading_days <= event_date][:10]

    # Fetch one
    company = yf.Ticker(ticker)
    sp = yf.Ticker("^GSPC")

    # Run in different day
    date_last = trading_days_before_event[0]
    event_date_adjust_history = trading_days_before_event[1] + timedelta(days=1) # history's end para is exclusive

    # Get Company's event date return
    df_company = company.history(start=date_last, end=event_date_adjust_history, auto_adjust=True)
    company_return = df_company["Close"].pct_change()
    company_return.dropna(inplace=True)

    # Get S&P500 event date return
    df_sp = sp.history(start=date_last, end=event_date_adjust_history, auto_adjust=True)  
    sp_return = df_sp["Close"].pct_change() # calculate Return for each day
    sp_return.dropna(inplace=True)


    if not company_return.empty and not sp_return.empty:
        company_return_list.append(company_return)
        sp_return_list.append(sp_return)


    # # Only past 10 days are needed 
    company_return_s = pd.concat(company_return_list)
    sp_return_s = pd.concat(sp_return_list)

    df_return = pd.DataFrame({
        "Company_return":company_return_s,
        "SP500_return":sp_return_s
    })
    

    # # Calcualte AR for each trade day
    abnormal_return_list = []
    company_return_data = df_return.iloc[0, 0]
    sp500_return_data = df_return.iloc[0, 1]

#     # Expected Return
    expected_return = alpha + beta * sp500_return_data

#     # Abnormal Return
    abnormal_return = company_return_data - expected_return

    abnormal_return_list.append(abnormal_return)

    event_ar_matrix.append(abnormal_return_list)


# column name
ar_columns = [f"AR"]

# # dataframe
df_ar_matrix = pd.DataFrame(event_ar_matrix, columns=ar_columns)

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta


df["Eventdate"] = pd.to_datetime(df["Eventdate"])

# 用 exchange_calendars 抓 NYSE 的交易日
nyse = get_calendar("NYSE")  

start_date = df["Eventdate"].min() - timedelta(days=5)
end_date = df["Eventdate"].max() + timedelta(days=5)

trading_schedule = nyse.schedule(start_date=start_date, end_date=end_date)
trading_days = trading_schedule.index.normalize()  

# 檢查每一個事件日是不是交易日
df["Is_Trading_Day"] = df["Eventdate"].isin(trading_days)

# 篩出不是交易日的事件
df_not_trading = df[df["Is_Trading_Day"] == False]

print(df_not_trading[["nid", "Ticker", "Eventdate"]])